In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
start = time.clock()
spark = SparkSession.builder.\
            master("local").\
            appName("amzone").\
            enableHiveSupport().\
            getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')
inputPath="data/sample.json"
# rawdata.select("reviewerID","asin","reviewerName","overall","reviewTime").show()
# inputPath = "data/reviews_Books_5.json"
rawdata = spark.read.json(inputPath)
ratingDf = rawdata.select(rawdata.reviewerID.alias("user"),\
               rawdata.asin.alias("item"),\
               rawdata.overall.alias("rating"))
ratingDf = ratingDf.select(ratingDf.user,ratingDf.item,ratingDf.rating.cast(DoubleType()))
ratingDf.registerTempTable("Ratings")

ParseException: u"\nextraneous input 'rating' expecting {')', ',', 'WHERE', 'GROUP', 'ORDER', 'HAVING', 'LIMIT', 'JOIN', 'CROSS', 'INNER', 'LEFT', 'RIGHT', 'FULL', 'NATURAL', 'LATERAL', 'WINDOW', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'SORT', 'CLUSTER', 'DISTRIBUTE', 'ANTI'}(line 1, pos 78)\n\n== SQL ==\nwith t1 as (select count(distinct(user)) as users,rating from Ratings groupBy rating),           t2 as (select count(distinct(item)) as items,rating from Ratings groupBy rating),             select * from t1 left join t2 on rating\n------------------------------------------------------------------------------^^^\n"

In [11]:
sqlString="with t1 as (select user,rating from Ratings group by user,rating) select * from t1"
#           t2 as (select count(distinct(item)) as items,rating from Ratings group by rating), \
#            select * from t1 left join t2 on t1.rating=t2.rating "
sqlString2 = "select count(*) as sm,rating from Ratings group by rating"
# rawdata.select("reviewerID").distinct().count()
resultDf = spark.sql(sqlString)
resultDf.show()

+--------+
|count(1)|
+--------+
|     100|
+--------+



In [ ]:
trainingData,cvData,testData = ratingDf.randomSplit([0.6,0.2,0.2])
# resultDf.show()
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(ratingDf)
itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(ratingDf)
alsModel = ALS(rank=10,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.1)
pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
pipeAlsModel = pipeAls.fit(trainingData)
predictionAls = pipeAlsModel.transform(trainingData)
predictionRecommendation= pipeAlsModel.transform(testData)
evaluator = RegressionEvaluator()\
      .setMetricName("rmse")\
      .setLabelCol("rating")\
      .setPredictionCol("prediction")
#rmse = evaluator.evaluate(predictionRecommendation.na.drop())
predictionAls.sort(asc("userid")).limit(10).show()
predictionRecommendation.sort(asc("userid")).limit(10).show()
#print "RMSE:",rmse
end = time.clock()
print "total:%f s" %(end-start)